# Prepare Model count

Uses:

- resources/historical.xlsx
- resources/databases_models.xlsx

Generates:

- resources/historical_join_db.xlsx


In [1]:
import sys  
sys.path.insert(1, '..')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR, HISTORICAL_FILE, HISTORICAL_FILE_JOIN_DB

In [3]:
#reads projects from Excel file
df = pd.read_excel(HISTORICAL_FILE, keep_default_na=False)
df.head()

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,ClickHouse,Kdb+,Virtuoso-NoSql,HBase,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df["DataBases"] = df["owner"] + "/" + df["name"]
columns_to_drop = ['owner', 'name', 'sha1', 'part_commit', 'date_commit', 'isLast', 'domain']
df = df.drop(columns=columns_to_drop)
df.head()

,SapHana,Realm,SAP SQL Anywhere,FileMaker,Virtuoso,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,...,Kdb+,Virtuoso-NoSql,HBase,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,DataBases
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Activiti/Activiti
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Activiti/Activiti
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Activiti/Activiti
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Activiti/Activiti
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Activiti/Activiti


In [5]:
df = df.groupby(['DataBases']).sum().reset_index().set_index('DataBases')
df.head()

,SapHana,Realm,SAP SQL Anywhere,FileMaker,Virtuoso,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,...,ClickHouse,Kdb+,Virtuoso-NoSql,HBase,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis
DataBases,,,,,,,,,,,,,,,,,,,,,
Activiti/Activiti,0,0,0,0,0,0,0,13,0,0,...,0,0,0,0,0,0,0,0,0,0
Alluxio/alluxio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,4,0,0
AlmasB/FXGL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AntennaPod/AntennaPod,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AnySoftKeyboard/AnySoftKeyboard,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
dftemp = df.T.copy()
# Drop multimodel db that appear twice
dftemp.drop(['Virtuoso', 'Ignite'], inplace=True)
df3 = dftemp.reset_index().rename(columns={'index': 'DataBases'})

df3.head()

DataBases,DataBases,Activiti/Activiti,Alluxio/alluxio,AlmasB/FXGL,AntennaPod/AntennaPod,AnySoftKeyboard/AnySoftKeyboard,AsyncHttpClient/async-http-client,Athou/commafeed,Atmosphere/atmosphere,Automattic/simplenote-android,...,vipshop/Saturn,vsch/flexmark-java,webbukkit/dynmap,wildfly/wildfly,winder/Universal-G-Code-Sender,xuxueli/xxl-job,yacy/yacy_search_server,zaproxy/zaproxy,zendesk/maxwell,zxing/zxing
0,SapHana,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Realm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SAP SQL Anywhere,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,FileMaker,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ArangoDB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
DATABASES_MODELS = RESOURCE_DIR + os.sep + 'databases_models.xlsx'
#reads projects from Excel file
df4 = pd.read_excel(DATABASES_MODELS, keep_default_na=False) #add in util.py
df4.head()

,DataBases,Primary Model,Second Model,Model
0,Aerospike,Key-value,Multi-model,NoSQL
1,ArangoDB,Multi-Model,,NoSQL
2,Cassandra,Wide column,,NoSQL
3,ClickHouse,Relational,Multi-model,SQL
4,Couchbase,Document,Multi-model,NoSQL


In [8]:
def check():
    itens_presentes = df3['DataBases'].isin(df4['DataBases'])

    nomes_presentes = df3[itens_presentes]

    # Filtrar os itens que estão faltando
    nomes_faltando = df3[~itens_presentes]

    # Converter as colunas para sets
    names_set = set(df4['DataBases'])
    projects_set = set(df3['DataBases'])

    # Itens presentes em 'name' mas não em 'projects'
    names_faltando = names_set - projects_set
    print("\nItens presentes em 'DF4' mas não em 'DF3':", names_faltando)

    # Itens presentes em 'projects' mas não em 'name'
    projects_faltando = projects_set - names_set
    print("Itens presentes em 'DF3' mas não em 'DF4':", projects_faltando)
check()


Itens presentes em 'DF4' mas não em 'DF3': {'Virtuoso_Sql', 'MS_SQL_Server', 'IBM_DB2', 'GoogleCloudFilestore', 'Virtuoso_NoSql', 'Ignite_NoSql', 'MS_Access', 'Ignite_Sql'}
Itens presentes em 'DF3' mas não em 'DF4': {'Virtuoso-NoSql', 'Ignite-NoSql', 'Virtuoso-Sql', 'MS Access', 'Ignite-Sql', 'GoogleCloudFirestore', 'MS SQL Server_Microsoft Azure SQL Database', 'IBM DB2'}


In [9]:
def renomear(item):
    if item == 'Ignite-NoSql':
        return 'Ignite_NoSql'
    elif item == 'IBM DB2':
        return 'IBM_DB2'
    elif item == 'MS Access':
        return 'MS_Access'
    elif item == 'Ignite-Sql':
        return 'Ignite_Sql'
    elif item == 'Virtuoso-NoSql':
        return 'Virtuoso_NoSql'
    elif item == 'Virtuoso-Sql':
        return 'Virtuoso_Sql'
    elif item == 'MS SQL Server_Microsoft Azure SQL Database':
        return 'MS_SQL_Server'
    else:
        return item

df3['DataBases'] = df3['DataBases'].apply(renomear)

print("\nDepois:")
df3.head()


Depois:


DataBases,DataBases,Activiti/Activiti,Alluxio/alluxio,AlmasB/FXGL,AntennaPod/AntennaPod,AnySoftKeyboard/AnySoftKeyboard,AsyncHttpClient/async-http-client,Athou/commafeed,Atmosphere/atmosphere,Automattic/simplenote-android,...,vipshop/Saturn,vsch/flexmark-java,webbukkit/dynmap,wildfly/wildfly,winder/Universal-G-Code-Sender,xuxueli/xxl-job,yacy/yacy_search_server,zaproxy/zaproxy,zendesk/maxwell,zxing/zxing
0,SapHana,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Realm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SAP SQL Anywhere,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,FileMaker,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ArangoDB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
check()


Itens presentes em 'DF4' mas não em 'DF3': {'GoogleCloudFilestore'}
Itens presentes em 'DF3' mas não em 'DF4': {'GoogleCloudFirestore'}


In [11]:
# Merging databases with data models
df_merge= pd.merge(df4, df3, left_on= 'DataBases', right_on='DataBases')
df_merge.rename(columns={'DataBases': 'Databases'}, inplace=True)
df_merge.to_excel(HISTORICAL_FILE_JOIN_DB, index=False) 
df_merge.head()


,Databases,Primary Model,Second Model,Model,Activiti/Activiti,Alluxio/alluxio,AlmasB/FXGL,AntennaPod/AntennaPod,AnySoftKeyboard/AnySoftKeyboard,AsyncHttpClient/async-http-client,...,vipshop/Saturn,vsch/flexmark-java,webbukkit/dynmap,wildfly/wildfly,winder/Universal-G-Code-Sender,xuxueli/xxl-job,yacy/yacy_search_server,zaproxy/zaproxy,zendesk/maxwell,zxing/zxing
0,Aerospike,Key-value,Multi-model,NoSQL,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ArangoDB,Multi-Model,,NoSQL,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cassandra,Wide column,,NoSQL,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ClickHouse,Relational,Multi-model,SQL,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Couchbase,Document,Multi-model,NoSQL,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
